In [1]:
# --- 1. INSTALAÇÃO DE BIBLIOTECAS ---
print("--- Instalando bibliotecas necessárias (pm4py)... ---")
!pip install pm4py

# Em alguns ambientes Colab, pode ser necessário instalar o graphviz
# !apt-get install graphviz -y

print("\n--- 2. IMPORTAÇÃO DE MÓDULOS ---")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import os
import shutil
import zipfile
from google.colab import files
import missingno as msno
import networkx as nx

# Módulos específicos de Process Mining
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.util import xes_constants

print("--- Bibliotecas importadas com sucesso. ---")

# --- 3. CARREGAMENTO DOS DADOS (UPLOAD) ---
print("\n--- 3. Por favor, carregue os 5 ficheiros CSV (projects, tasks, resources, resource_allocations, dependencies) ---")
try:
    # Abre a janela de upload do Colab
    uploaded = files.upload()

    # Verifica se os ficheiros essenciais foram carregados
    required_files = ['projects.csv', 'tasks.csv', 'resources.csv', 'resource_allocations.csv', 'dependencies.csv']
    for f in required_files:
        if f not in uploaded:
            raise FileNotFoundError(f"O ficheiro '{f}' não foi encontrado nos ficheiros carregados.")

    print("\n--- Ficheiros carregados com sucesso! ---")

    # Carregar para DataFrames
    df_projects_orig = pd.read_csv('projects.csv')
    df_tasks_orig = pd.read_csv('tasks.csv')
    df_resources_orig = pd.read_csv('resources.csv')
    df_resource_allocations_orig = pd.read_csv('resource_allocations.csv')
    df_dependencies_orig = pd.read_csv('dependencies.csv')

    print("--- DataFrames criados a partir dos CSVs. ---")

except FileNotFoundError as e:
    print(f"\nERRO: {e}")
    print("A execução foi interrompida. Por favor, execute a célula novamente e carregue todos os ficheiros necessários.")
    exit()
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")
    exit()

# --- 4. PRÉ-PROCESSAMENTO E ENGENHARIA DE FUNCIONALIDADES ---
print("\n--- 4. Iniciando pré-processamento e engenharia de funcionalidades... ---")
# Criar cópias para evitar alterar os dataframes originais carregados
df_projects = df_projects_orig.copy()
df_tasks = df_tasks_orig.copy()
df_resources = df_resources_orig.copy()
df_resource_allocations = df_resource_allocations_orig.copy()
df_dependencies = df_dependencies_orig.copy()

# Conversões de data
for df in [df_projects, df_tasks]:
    for col in ['start_date', 'end_date', 'planned_end_date']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

# Features a nível de projeto
df_projects['days_diff'] = (df_projects['end_date'] - df_projects['planned_end_date']).dt.days
df_projects['actual_duration_days'] = (df_projects['end_date'] - df_projects['start_date']).dt.days
df_projects['project_type'] = df_projects['project_name'].str.extract(r'Projeto \d+: (.*?) ')
df_projects['completion_month'] = df_projects['end_date'].dt.to_period('M')
df_projects['completion_quarter'] = df_projects['end_date'].dt.to_period('Q')

# Features a nível de tarefa
df_tasks['task_duration_days'] = (df_tasks['end_date'] - df_tasks['start_date']).dt.days

# Agregações para métricas de projeto (custo, recursos, complexidade)
df_alloc_costs = df_resource_allocations.merge(df_resources, on='resource_id')
df_alloc_costs['cost_of_work'] = df_alloc_costs['hours_worked'] * df_alloc_costs['cost_per_hour']

dep_counts = df_dependencies.groupby('project_id').size().reset_index(name='dependency_count')
task_counts = df_tasks.groupby('project_id').size().reset_index(name='task_count')
project_complexity = pd.merge(dep_counts, task_counts, on='project_id', how='outer').fillna(0)
project_complexity['complexity_ratio'] = (project_complexity['dependency_count'] / project_complexity['task_count']).fillna(0)

project_aggregates = df_alloc_costs.groupby('project_id').agg(
    total_actual_cost=('cost_of_work', 'sum'),
    avg_hourly_rate=('cost_per_hour', 'mean'),
    num_resources=('resource_id', 'nunique')
).reset_index()

df_projects = df_projects.merge(project_aggregates, on='project_id', how='left')
df_projects = df_projects.merge(project_complexity, on='project_id', how='left')
df_projects['cost_diff'] = df_projects['total_actual_cost'] - df_projects['budget_impact']
df_projects['cost_per_day'] = df_projects['total_actual_cost'] / df_projects['actual_duration_days'].replace(0, np.nan)

# DataFrame Unificado para análises cruzadas
df_full_context = df_tasks.merge(df_projects, on='project_id', suffixes=('_task', '_project'))
allocations_to_merge = df_resource_allocations.drop(columns=['project_id'], errors='ignore')
df_full_context = df_full_context.merge(allocations_to_merge, on='task_id')
df_full_context = df_full_context.merge(df_resources, on='resource_id')
df_full_context['cost_of_work'] = df_full_context['hours_worked'] * df_full_context['cost_per_hour']
print("--- DataFrames enriquecidos foram criados. ---")

# --- 5. CRIAÇÃO DO LOG DE EVENTOS PARA PROCESS MINING ---
print("\n--- 5. Criando Log de Eventos para Process Mining... ---")
# Criar eventos de 'start'
df_start_events = df_tasks[['project_id', 'task_id', 'task_name', 'start_date']].copy()
df_start_events.rename(columns={'start_date': 'time:timestamp', 'task_name': 'concept:name', 'project_id': 'case:concept:name'}, inplace=True)
df_start_events['lifecycle:transition'] = 'start'

# Criar eventos de 'complete'
df_complete_events = df_tasks[['project_id', 'task_id', 'task_name', 'end_date']].copy()
df_complete_events.rename(columns={'end_date': 'time:timestamp', 'task_name': 'concept:name', 'project_id': 'case:concept:name'}, inplace=True)
df_complete_events['lifecycle:transition'] = 'complete'

# Juntar para adicionar recursos
log_df_temp = pd.concat([df_start_events, df_complete_events], ignore_index=True)
# Para simplificar, atribuímos todos os recursos de uma tarefa a ambos os eventos (start/complete)
resource_mapping = df_resource_allocations.groupby('task_id')['resource_id'].apply(list).reset_index()
log_df_temp = log_df_temp.merge(resource_mapping, on='task_id', how='left').explode('resource_id')
log_df_temp = log_df_temp.merge(df_resources[['resource_id', 'resource_name']], on='resource_id', how='left')
log_df_temp.rename(columns={'resource_name': 'org:resource'}, inplace=True)

# Limpeza e formatação final do log
log_df_final = log_df_temp[['case:concept:name', 'concept:name', 'time:timestamp', 'lifecycle:transition', 'org:resource']].copy()
log_df_final.dropna(subset=['time:timestamp', 'org:resource'], inplace=True)
log_df_final['case:concept:name'] = log_df_final['case:concept:name'].astype(str)
log_df_final.sort_values(['case:concept:name', 'time:timestamp'], inplace=True)

# Converter para o objeto de log do pm4py
event_log_pm4py = log_converter.apply(log_df_final)

print("--- Log de Eventos criado e pronto para análise. ---")
print(f"\nTOTAL DE EVENTOS NO LOG: {len(log_df_final)}")
print(f"NÚMERO DE CASOS (PROJETOS): {log_df_final['case:concept:name'].nunique()}")
print("\n✅ CÉLULA 1 CONCLUÍDA: Ambiente, dados e log de eventos estão prontos.")

--- Instalando bibliotecas necessárias (pm4py)... ---
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=f31d071a9a91d218097b86589ad57c8fe48d3a70276bb71774d1a6a1adcec6f8
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa74a8e16e9585896e83
Successfully built intervaltree

--- 2. IMPORTAÇÃO DE MÓDULOS ---


/usr/local/lib/python3.12/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.12/dist-packages/cudf/pandas/__init__.py:64: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


--- Bibliotecas importadas com sucesso. ---

--- 3. Por favor, carregue os 5 ficheiros CSV (projects, tasks, resources, resource_allocations, dependencies) ---


Saving dependencies.csv to dependencies.csv
Saving projects.csv to projects.csv
Saving resource_allocations.csv to resource_allocations.csv
Saving resources.csv to resources.csv
Saving tasks.csv to tasks.csv

--- Ficheiros carregados com sucesso! ---
--- DataFrames criados a partir dos CSVs. ---

--- 4. Iniciando pré-processamento e engenharia de funcionalidades... ---
--- DataFrames enriquecidos foram criados. ---

--- 5. Criando Log de Eventos para Process Mining... ---
--- Log de Eventos criado e pronto para análise. ---

TOTAL DE EVENTOS NO LOG: 938
NÚMERO DE CASOS (PROJETOS): 50

✅ CÉLULA 1 CONCLUÍDA: Ambiente, dados e log de eventos estão prontos.


In [ ]:
# --- INÍCIO DA CÉLULA 2: Painel de Análise de Processos Completo e Definitivo ---
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
# Assumindo que 'files' está disponível no seu ambiente (como Google Colab)
# from google.colab import files

print("--- Executando Célula 2: Geração do Painel de Análise Completo ---")

# --- CORREÇÕES DEFINITIVAS APLICADAS AQUI ---
# Este bloco garante a robustez do script, prevenindo erros de compatibilidade e de tipos de dados.

# 1. Corrigir tipos de dados de período (mensal, trimestral, etc.) para evitar erros com o Seaborn.
#    A conversão para string resolve os `ValueError: Data type period[...] not supported`.
if 'completion_month' in df_projects.columns:
    df_projects['completion_month'] = df_projects['completion_month'].astype(str)
if 'completion_quarter' in df_projects.columns:
    df_projects['completion_quarter'] = df_projects['completion_quarter'].astype(str)

# 2. Garantir que as colunas de ID usadas para merges sejam consistentes (string).
#    Isto previne futuros `ValueError` de incompatibilidade de tipos (int64 vs object).
df_projects['project_id'] = df_projects['project_id'].astype(str)
if 'project_id' in df_tasks.columns:
    df_tasks['project_id'] = df_tasks['project_id'].astype(str)
if 'case:concept:name' in log_df_final.columns:
    log_df_final['case:concept:name'] = log_df_final['case:concept:name'].astype(str)
if 'project_id' in df_full_context.columns:
    df_full_context['project_id'] = df_full_context['project_id'].astype(str)
# --- FIM DAS CORREÇÕES ---


# --- 1. SETUP DO RELATÓRIO E EXPORTAÇÃO ---
report_dir_analysis = 'Process_Analysis_Dashboard'
plots_dir_analysis = os.path.join(report_dir_analysis, 'plots')
data_dir_analysis = os.path.join(report_dir_analysis, 'data')

if os.path.exists(report_dir_analysis): shutil.rmtree(report_dir_analysis)
os.makedirs(plots_dir_analysis, exist_ok=True); os.makedirs(data_dir_analysis, exist_ok=True)

report_path_analysis = os.path.join(report_dir_analysis, 'Analysis_Report.html')
html_header = """
<!DOCTYPE html><html lang="pt"><head><meta charset="UTF-8"><title>Painel de Análise de Processos</title><style>
body {{ font-family: sans-serif; margin: 0; background-color: #f4f4f9; color: #333; }}
.container {{ max-width: 1200px; margin: auto; padding: 20px; background-color: #fff; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
h1, h2, h3 {{ color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px; }}
h1 {{ font-size: 2.5em; text-align: center; }} h2 {{ font-size: 2em; margin-top: 40px; }} h3 {{ font-size: 1.5em; border-bottom: 1px solid #ccc; }}
img {{ max-width: 90%; height: auto; display: block; margin: 20px auto; border: 1px solid #ddd; border-radius: 5px; }}
p, li {{ line-height: 1.6; }} table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
th, td {{ padding: 12px; border: 1px solid #ddd; text-align: left; }} th {{ background-color: #3498db; color: white; }}
tr:nth-child(even) {{ background-color: #f2f2f2; }}
</style></head><body><div class="container">
<h1>Painel de Análise de Processos</h1><p>Relatório gerado em: {date}</p>
""".format(date=pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'))

with open(report_path_analysis, 'w', encoding='utf-8') as f: f.write(html_header)

def save_plot_and_add_to_html_report(title, content, plot_filename, is_matplotlib=False, fig=None):
    full_path = os.path.join(plots_dir_analysis, plot_filename)
    if is_matplotlib and fig is not None:
        fig.savefig(full_path, bbox_inches='tight')
    else:
        plt.savefig(full_path, bbox_inches='tight')
    plt.close('all')
    with open(report_path_analysis, 'a', encoding='utf-8') as f:
        f.write(f"<h3>{title}</h3>\n<p>{content}</p>\n<img src='plots/{plot_filename}' alt='{title}'>\n")
    print(f"Plot '{plot_filename}' salvo.")

plot_counter = 0

# --- Secção 1: Análises de Alto Nível e de Casos ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 1: Análises de Alto Nível e de Casos</h2>")
monthly_throughput = df_projects.groupby(df_projects['end_date'].dt.to_period('M').astype(str)).size().mean()
kpi_data = {
    'Métrica': ['Total de Projetos (Casos)', 'Total de Tarefas', 'Total de Eventos no Log', 'Total de Recursos Únicos', 'Duração Média dos Projetos (dias)', 'Produtividade Média (Projetos/Mês)'],
    'Valor': [df_projects['project_id'].nunique(), len(df_tasks), len(log_df_final), df_resources['resource_id'].nunique(), df_projects['actual_duration_days'].mean(), monthly_throughput]
}
kpi_df = pd.DataFrame(kpi_data)
kpi_styled = kpi_df.style.format({'Valor': '{:,.2f}'}).set_properties(**{'text-align': 'right'})
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h3>Painel de KPIs de Alto Nível</h3>" + kpi_styled.to_html(index=False))

plot_counter += 1; filename = f"plot_{plot_counter:02d}_performance_matrix.png"
plt.figure(figsize=(12, 8)); sns.scatterplot(data=df_projects, x='days_diff', y='cost_diff', hue='project_type', s=100, alpha=0.7, palette='bright'); plt.axhline(0, color='black', linestyle='--', lw=1); plt.axvline(0, color='black', linestyle='--', lw=1);
save_plot_and_add_to_html_report('Matriz de Performance: Prazo vs. Orçamento', 'Cada ponto é um projeto, classificado pela sua performance em relação ao prazo e ao orçamento. As linhas definem os quadrantes.', filename)

plot_counter += 1; filename = f"plot_{plot_counter:02d}_case_durations_boxplot.png"
plt.figure(figsize=(12, 7)); sns.boxplot(x=df_projects['actual_duration_days'], color='skyblue'); sns.stripplot(x=df_projects['actual_duration_days'], color='blue', size=4, jitter=True, alpha=0.5);
save_plot_and_add_to_html_report('Distribuição da Duração dos Projetos (Lead Time)', 'Analisa a distribuição do tempo total de execução dos projetos, ajudando a identificar a média, a variabilidade e os outliers.', filename)

outlier_duration = df_projects.sort_values('actual_duration_days', ascending=False).head(5)
outlier_cost = df_projects.sort_values('total_actual_cost', ascending=False).head(5)
outlier_duration_styled = outlier_duration[['project_name', 'actual_duration_days', 'total_actual_cost', 'days_diff']].style.format({'actual_duration_days': '{:,.0f}','total_actual_cost': '{:,.0f}€', 'days_diff': '{:,.0f}'}).set_properties(**{'text-align': 'right'})
outlier_cost_styled = outlier_cost[['project_name', 'total_actual_cost', 'actual_duration_days', 'cost_diff']].style.format({'total_actual_cost': '{:,.0f}€', 'actual_duration_days': '{:,.0f}', 'cost_diff': '{:,.0f}€'}).set_properties(**{'text-align': 'right'})
with open(report_path_analysis, 'a', encoding='utf-8') as f:
    f.write("<h3>Top 5 Projetos Outliers por Duração</h3>" + outlier_duration_styled.to_html(index=False))
    f.write("<h3>Top 5 Projetos Outliers por Custo</h3>" + outlier_cost_styled.to_html(index=False))

# --- Secção 2: Análises de Performance Detalhada ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 2: Análises de Performance Detalhada</h2>")
lead_times = log_df_final.groupby("case:concept:name")["time:timestamp"].agg(["min", "max"]).reset_index()
lead_times["lead_time_days"] = (lead_times["max"] - lead_times["min"]).dt.total_seconds() / (24*60*60)
def compute_avg_throughput(group):
    group = group.sort_values("time:timestamp"); deltas = group["time:timestamp"].diff().dropna()
    return deltas.mean().total_seconds() if not deltas.empty else 0
# Correção do DeprecationWarning: remover 'group_keys=False'
throughput_per_case = log_df_final.groupby("case:concept:name").apply(compute_avg_throughput).reset_index(name="avg_throughput_seconds")
throughput_per_case["avg_throughput_hours"] = throughput_per_case["avg_throughput_seconds"] / 3600
perf_df = pd.merge(lead_times, throughput_per_case, on="case:concept:name")
perf_df.to_csv(os.path.join(data_dir_analysis, "performance_metrics.csv"), index=False)
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h3>Tabela de Estatísticas de Lead Time e Throughput</h3>" + perf_df[["lead_time_days", "avg_throughput_hours"]].describe().to_html())
plot_counter += 1; filename = f"plot_{plot_counter:02d}_lead_time_hist.png"; plt.figure(figsize=(10,4)); sns.histplot(perf_df["lead_time_days"], bins=30, kde=True);
save_plot_and_add_to_html_report('Distribuição do Lead Time por Caso (dias)', 'Histograma do tempo total de execução de cada projeto.', filename)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_throughput_hist.png"; plt.figure(figsize=(10,4)); sns.histplot(perf_df["avg_throughput_hours"], bins=30, kde=True, color='green');
save_plot_and_add_to_html_report('Distribuição do Throughput (horas)', 'Histograma do tempo médio entre a conclusão de tarefas consecutivas em cada projeto.', filename)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_throughput_boxplot.png"
plt.figure(figsize=(10, 6)); sns.boxplot(x=perf_df["avg_throughput_hours"], color='lightgreen'); sns.stripplot(x=perf_df["avg_throughput_hours"], color='green', size=4, jitter=True, alpha=0.5);
save_plot_and_add_to_html_report("Boxplot do Throughput por Caso (horas)", "Visualização alternativa da distribuição do Throughput.", filename)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_lead_time_vs_throughput.png"; plt.figure(figsize=(10, 6)); sns.regplot(x="avg_throughput_hours", y="lead_time_days", data=perf_df);
save_plot_and_add_to_html_report("Relação entre Lead Time e Throughput", "Analisa se um maior tempo entre tarefas (throughput) está correlacionado com um maior tempo total do projeto (lead time).", filename)

# --- Secção 3: Análise de Atividades e Handoffs ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 3: Análise de Atividades e Handoffs</h2>")
service_times = df_full_context.groupby('task_name')['hours_worked'].mean().reset_index()
service_times['service_time_days'] = service_times['hours_worked'] / 8
plot_counter += 1; filename = f"plot_{plot_counter:02d}_activity_service_times.png"; plt.figure(figsize=(12, 8)); sns.barplot(x='service_time_days', y='task_name', data=service_times.sort_values('service_time_days', ascending=False).head(10), hue='task_name', palette='viridis', legend=False);
save_plot_and_add_to_html_report('Tempo Médio de Execução por Atividade', 'Mostra as atividades que, em média, demoram mais tempo de trabalho ativo.', filename)
df_handoff_analysis = log_df_final[log_df_final['lifecycle:transition'] == 'complete'].copy()
df_handoff_analysis.sort_values(['case:concept:name', 'time:timestamp'], inplace=True)
df_handoff_analysis['previous_activity_end_time'] = df_handoff_analysis.groupby('case:concept:name')['time:timestamp'].shift(1)
df_handoff_analysis['handoff_time_days'] = (df_handoff_analysis['time:timestamp'] - df_handoff_analysis['previous_activity_end_time']).dt.total_seconds() / (24*3600)
df_handoff_analysis['previous_activity'] = df_handoff_analysis.groupby('case:concept:name')['concept:name'].shift(1)
handoff_stats = df_handoff_analysis.groupby(['previous_activity', 'concept:name'])['handoff_time_days'].mean().reset_index().sort_values('handoff_time_days', ascending=False)
handoff_stats.to_csv(os.path.join(data_dir_analysis, "handoff_times_manual.csv"), index=False)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_top_handoffs.png"; plt.figure(figsize=(12, 8)); handoff_stats['transition'] = handoff_stats['previous_activity'].fillna('') + ' -> ' + handoff_stats['concept:name'].fillna(''); sns.barplot(data=handoff_stats.head(10), y='transition', x='handoff_time_days', hue='transition', palette='magma', legend=False);
save_plot_and_add_to_html_report('Top 10 Transições com Maior Tempo de Espera (Handoff)', 'Identifica os maiores gargalos de tempo de espera entre tarefas consecutivas.', filename)
avg_project_cost_per_day = df_projects['cost_per_day'].mean()
handoff_stats['estimated_cost_of_wait'] = handoff_stats['handoff_time_days'] * avg_project_cost_per_day
plot_counter += 1; filename = f"plot_{plot_counter:02d}_top_handoffs_cost.png"; plt.figure(figsize=(12, 8)); sns.barplot(data=handoff_stats.sort_values('estimated_cost_of_wait', ascending=False).head(10), y='transition', x='estimated_cost_of_wait', hue='transition', palette='Reds_r', legend=False);
save_plot_and_add_to_html_report('Top 10 Transições por Custo de Espera Estimado', 'Quantifica o impacto financeiro dos tempos de espera, multiplicando o tempo de handoff pelo custo médio diário de um projeto.', filename)

# --- Secção 4: Análise Organizacional (Recursos) ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 4: Análise Organizacional (Recursos)</h2>")
activity_counts = df_tasks["task_name"].value_counts()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_top_activities_plot.png"; plt.figure(figsize=(10,4)); sns.barplot(x=activity_counts.head(10).values, y=activity_counts.head(10).index);
save_plot_and_add_to_html_report("Atividades Mais Frequentes", 'Mostra as tarefas que ocorrem com maior frequência em todos os projetos.', filename)
resource_workload = df_full_context.groupby('resource_name')['hours_worked'].sum().sort_values(ascending=False).reset_index()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_resource_workload.png"; plt.figure(figsize=(12, 8)); sns.barplot(x='hours_worked', y='resource_name', data=resource_workload.head(10), hue='resource_name', palette='plasma', legend=False);
save_plot_and_add_to_html_report('Top 10 Recursos por Horas Trabalhadas', 'Recursos com maior carga de trabalho total.', filename)
resource_metrics = df_full_context.groupby("resource_name").agg(unique_cases=('project_id', 'nunique'), event_count=('task_id', 'count')).reset_index()
resource_metrics["avg_events_per_case"] = resource_metrics["event_count"] / resource_metrics["unique_cases"]
plot_counter += 1; filename = f"plot_{plot_counter:02d}_resource_avg_events.png"; plt.figure(figsize=(12, 8)); sns.barplot(x='avg_events_per_case', y='resource_name', data=resource_metrics.sort_values('avg_events_per_case', ascending=False).head(10), hue='resource_name', palette='coolwarm', legend=False);
save_plot_and_add_to_html_report('Top 10 Recursos por Média de Tarefas por Projeto', 'Identifica os recursos mais envolvidos no processo, medido pelo número médio de tarefas por projeto.', filename)
resource_activity_matrix_pivot = df_full_context.pivot_table(index='resource_name', columns='task_name', values='hours_worked', aggfunc='sum').fillna(0)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_resource_activity_matrix.png"; plt.figure(figsize=(15, 10)); sns.heatmap(resource_activity_matrix_pivot, cmap='YlGnBu', annot=True, fmt=".0f");
save_plot_and_add_to_html_report('Heatmap de Esforço (Horas) por Recurso e Atividade', 'Mostra o total de horas que cada recurso dedicou a cada tarefa.', filename)
handoff_counts = {};
for trace in event_log_pm4py:
    resources = [event['org:resource'] for event in trace if 'org:resource' in event]
    for i in range(len(resources) - 1):
        pair = (resources[i], resources[i+1])
        if resources[i] != resources[i+1]:
            handoff_counts[pair] = handoff_counts.get(pair, 0) + 1
df_resource_handoffs = pd.DataFrame([{'De': k[0], 'Para': k[1], 'Contagem': v} for k, v in handoff_counts.items()]).sort_values('Contagem', ascending=False)
df_resource_handoffs.to_csv(os.path.join(data_dir_analysis, 'resource_handoffs.csv'), index=False)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_resource_handoffs.png"; plt.figure(figsize=(16, 9)); plt.subplots_adjust(left=0.45); df_resource_handoffs['Handoff'] = df_resource_handoffs['De'] + ' -> ' + df_resource_handoffs['Para']; sns.barplot(x='Contagem', y='Handoff', data=df_resource_handoffs.head(10), hue='Handoff', palette='rocket', legend=False);
save_plot_and_add_to_html_report('Top 10 Handoffs entre Recursos', 'Mostra as transferências de trabalho mais frequentes entre diferentes pessoas.', filename)

# --- GRÁFICO ADICIONADO 1 (CORRIGIDO): Principais Direcionadores de Custo por Tipo de Recurso ---
cost_by_resource_type = df_full_context.groupby('resource_type')['cost_of_work'].sum().sort_values(ascending=False).reset_index()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_cost_by_resource_type.png"
plt.figure(figsize=(10, 6))
sns.barplot(data=cost_by_resource_type, x='cost_of_work', y='resource_type', hue='resource_type', palette='magma', legend=False)
save_plot_and_add_to_html_report('Principais Direcionadores de Custo por Tipo de Recurso', 'Análise do custo total gerado por cada tipo de recurso no processo.', filename)

# --- Secção 5: Análise de Variantes e Rework ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 5: Análise de Variantes e Rework</h2>")
variants_df = log_df_final[log_df_final['lifecycle:transition'] == 'complete'].groupby('case:concept:name')['concept:name'].apply(list).reset_index(name='trace')
variants_df['variant_str'] = variants_df['trace'].apply(lambda x: ' -> '.join(x))
variant_analysis = variants_df['variant_str'].value_counts().reset_index(name='frequency')
variant_analysis['percentage'] = (variant_analysis['frequency'] / variant_analysis['frequency'].sum()) * 100
variant_analysis_styled = variant_analysis.head(10).style.format({'frequency': '{:,.0f}', 'percentage': '{:.2f}%'}).set_properties(**{'text-align': 'right'})
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h3>Tabela das Top 10 Variantes por Frequência</h3>" + variant_analysis_styled.to_html(index=False))
plot_counter += 1; filename = f"plot_{plot_counter:02d}_variants_frequency.png"; plt.figure(figsize=(16, 9)); plt.subplots_adjust(left=0.45); sns.barplot(x='frequency', y='variant_str', data=variant_analysis.head(10), hue='variant_str', palette='coolwarm', legend=False);
save_plot_and_add_to_html_report('Top 10 Variantes de Processo por Frequência', 'Visualização dos caminhos mais comuns que os projetos seguem.', filename)
rework_loops = Counter()
for trace in variants_df['trace']:
    for i in range(len(trace) - 2):
        if trace[i] == trace[i+2] and trace[i] != trace[i+1]: rework_loops[f"{trace[i]} -> {trace[i+1]} -> {trace[i]}"] += 1
df_rework = pd.DataFrame(rework_loops.most_common(10), columns=['rework_loop', 'frequency'])
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h3>Principais Loops de Rework Identificados</h3><p>Indica retrabalho, uma fonte primária de ineficiência.</p>" + df_rework.to_html(index=False))

# --- Secção 6: Análise Aprofundada (Causa-Raiz, Financeira e Benchmarking) ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h2>Secção 6: Análise Aprofundada (Causa-Raiz, Financeira e Benchmarking)</h2>")
# Custo do Atraso
delayed_projects = df_projects[df_projects['days_diff'] > 0]
cost_of_delay_kpis = { 'Métrica': ['Custo Total de Projetos Atrasados', 'Atraso Médio (dias)', 'Custo Médio por Dia de Atraso'],
                       'Valor': [delayed_projects['total_actual_cost'].sum(), delayed_projects['days_diff'].mean(), (delayed_projects[delayed_projects['days_diff'] > 0]['total_actual_cost'] / delayed_projects[delayed_projects['days_diff'] > 0]['days_diff']).mean()]}
cost_of_delay_kpis_styled = pd.DataFrame(cost_of_delay_kpis).style.format({'Valor': '{:,.0f}€'}).set_properties(**{'text-align': 'right'})
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("<h3>Análise de Custo do Atraso</h3>" + cost_of_delay_kpis_styled.to_html(index=False))
# Impacto do Tamanho da Equipa no Atraso
min_res, max_res = df_projects['num_resources'].min(), df_projects['num_resources'].max()
bins = np.linspace(min_res, max_res, 5, dtype=int) if max_res > min_res else [min_res, max_res]
df_projects['team_size_bin_dynamic'] = pd.cut(df_projects['num_resources'], bins=bins, include_lowest=True, duplicates='drop').astype(str)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_delay_by_teamsize.png"; plt.figure(figsize=(12, 7)); sns.boxplot(data=df_projects.dropna(subset=['team_size_bin_dynamic']), x='team_size_bin_dynamic', y='days_diff', hue='team_size_bin_dynamic', palette='flare', legend=False);
save_plot_and_add_to_html_report('Impacto do Tamanho da Equipa no Atraso', 'Compara a distribuição do atraso para projetos com diferentes tamanhos de equipa.', filename)

# --- GRÁFICO ADICIONADO 2: Benchmark de Duração Mediana por Tamanho da Equipa ---
median_duration_by_team_size = df_projects.groupby('team_size_bin_dynamic')['actual_duration_days'].median().reset_index()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_median_duration_by_teamsize.png"
plt.figure(figsize=(10, 7))
sns.barplot(data=median_duration_by_team_size, x='team_size_bin_dynamic', y='actual_duration_days', hue='team_size_bin_dynamic', palette='crest', legend=False)
save_plot_and_add_to_html_report(
    'Benchmark de Duração Mediana por Tamanho da Equipa',
    'Compara a performance de projetos com diferentes tamanhos de equipa.',
    filename
)

# Análise de Eficiência Semanal
df_alloc_costs['day_of_week'] = pd.to_datetime(df_alloc_costs['allocation_date']).dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
plot_counter += 1; filename = f"plot_{plot_counter:02d}_weekly_efficiency.png"; plt.figure(figsize=(12, 7)); sns.barplot(data=df_alloc_costs.groupby('day_of_week')['hours_worked'].sum().reindex(weekday_order).reset_index(), x='day_of_week', y='hours_worked', hue='day_of_week', palette='plasma', legend=False);
save_plot_and_add_to_html_report('Análise de Eficiência Semanal', 'Distribuição do total de horas trabalhadas por dia da semana.', filename)

# Análise de Gargalos Aprofundada
df_tasks_analysis = df_tasks.copy(); df_tasks_analysis['service_time_days'] = (pd.to_datetime(df_tasks_analysis['end_date']) - pd.to_datetime(df_tasks_analysis['start_date'])).dt.total_seconds() / (24*60*60)
df_tasks_analysis.sort_values(['project_id', 'start_date'], inplace=True); df_tasks_analysis['previous_task_end'] = df_tasks_analysis.groupby('project_id')['end_date'].shift(1)
df_tasks_analysis['waiting_time_days'] = (pd.to_datetime(df_tasks_analysis['start_date']) - pd.to_datetime(df_tasks_analysis['previous_task_end'])).dt.total_seconds() / (24*60*60)
df_tasks_analysis['waiting_time_days'] = df_tasks_analysis['waiting_time_days'].apply(lambda x: x if x > 0 else 0)
df_tasks_with_resources = df_tasks_analysis.merge(df_full_context[['task_id', 'resource_name']], on='task_id', how='left').drop_duplicates()
bottleneck_by_resource = df_tasks_with_resources.groupby('resource_name')['waiting_time_days'].mean().sort_values(ascending=False).head(15).reset_index()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_bottleneck_by_resource.png"; plt.figure(figsize=(12, 8));
sns.barplot(data=bottleneck_by_resource, y='resource_name', x='waiting_time_days', hue='resource_name', palette='rocket', legend=False);
save_plot_and_add_to_html_report('Top 15 Recursos por Tempo Médio de Espera', 'Identifica os recursos individuais cujas tarefas ficam mais tempo em espera antes de serem iniciadas.', filename)
bottleneck_by_activity = df_tasks_analysis.groupby('task_type')[['service_time_days', 'waiting_time_days']].mean()

# --- GRÁFICO ADICIONADO 3: Análise de Gargalos (Tempo de Serviço vs. Tempo de Espera) ---
plot_counter += 1; filename = f"plot_{plot_counter:02d}_service_vs_wait_stacked.png"
fig, ax = plt.subplots(figsize=(12, 7))
bottleneck_by_activity.plot(kind='bar', stacked=True, color=['royalblue', 'crimson'], ax=ax)
ax.set_ylabel('Dias')
ax.set_xlabel('Tipo de Tarefa')
ax.tick_params(axis='x', rotation=45)
save_plot_and_add_to_html_report(
    'Análise de Gargalos (Tempo de Serviço vs. Tempo de Espera)',
    'Compara o tempo de trabalho ativo (azul) com o tempo de espera (vermelho).',
    filename,
    is_matplotlib=True,
    fig=fig
)

plot_counter += 1; filename = f"plot_{plot_counter:02d}_wait_vs_service_scatter.png"; plt.figure(figsize=(10, 6)); sns.regplot(data=bottleneck_by_activity, x='service_time_days', y='waiting_time_days');
save_plot_and_add_to_html_report("Tempo de Espera vs. Tempo de Execução", "Analisa se as tarefas que demoram mais a executar (eixo X) também são as que ficam mais tempo à espera (eixo Y).", filename)

df_wait_over_time = df_tasks_analysis.merge(df_projects[['project_id', 'completion_month']], on='project_id')
monthly_wait_time = df_wait_over_time.groupby('completion_month')['waiting_time_days'].mean().reset_index()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_wait_time_evolution.png"; plt.figure(figsize=(12, 6)); sns.lineplot(data=monthly_wait_time, x='completion_month', y='waiting_time_days', marker='o'); plt.xticks(rotation=45);
save_plot_and_add_to_html_report("Evolução do Tempo Médio de Espera", "Mostra a tendência do tempo de espera entre tarefas, mês a mês.", filename)

# NOVAS Análises de Mercado
# Matriz de Handoffs por Tipo de Equipa
df_rh_typed = df_resource_handoffs.merge(df_resources[['resource_name', 'resource_type']], left_on='De', right_on='resource_name').merge(df_resources[['resource_name', 'resource_type']], left_on='Para', right_on='resource_name', suffixes=('_de', '_para'))
handoff_matrix = df_rh_typed.groupby(['resource_type_de', 'resource_type_para'])['Contagem'].sum().unstack().fillna(0)
plot_counter += 1; filename = f"plot_{plot_counter:02d}_handoff_matrix_by_type.png"; plt.figure(figsize=(10, 8)); sns.heatmap(handoff_matrix, annot=True, fmt=".0f", cmap="BuPu");
save_plot_and_add_to_html_report("Matriz de Handoffs por Tipo de Equipa", "Mostra o volume de transferências de trabalho entre os diferentes tipos de equipas.", filename)

# Benchmark de Throughput por Tamanho da Equipa
df_perf_full = perf_df.merge(df_projects, left_on='case:concept:name', right_on='project_id')
plot_counter += 1; filename = f"plot_{plot_counter:02d}_throughput_benchmark_by_teamsize.png"; plt.figure(figsize=(12, 7)); sns.boxplot(data=df_perf_full, x='team_size_bin_dynamic', y='avg_throughput_hours', hue='team_size_bin_dynamic', palette='plasma', legend=False);
save_plot_and_add_to_html_report('Benchmark de Throughput por Tamanho da Equipa', 'Compara a velocidade interna do processo (tempo médio entre tarefas) para equipas de diferentes tamanhos.', filename)
# Análise de Ciclo por Fase do Processo
def get_phase(task_type):
    if task_type in ['Desenvolvimento', 'Correção', 'Revisão', 'Design']: return 'Desenvolvimento & Design'
    if task_type == 'Teste': return 'Teste (QA)'
    if task_type in ['Deploy', 'DBA']: return 'Operações & Deploy'
    return 'Outros'
df_tasks_phases = df_tasks.copy(); df_tasks_phases['phase'] = df_tasks_phases['task_type'].apply(get_phase)
phase_times = df_tasks_phases.groupby(['project_id', 'phase']).agg(start=('start_date', 'min'), end=('end_date', 'max')).reset_index()
phase_times['cycle_time_days'] = (phase_times['end'] - phase_times['start']).dt.days
avg_cycle_time_by_phase = phase_times.groupby('phase')['cycle_time_days'].mean()
plot_counter += 1; filename = f"plot_{plot_counter:02d}_cycle_time_breakdown.png"; plt.figure(figsize=(10, 6)); avg_cycle_time_by_phase.plot(kind='bar', color=sns.color_palette('muted'));
save_plot_and_add_to_html_report('Duração Média por Fase do Processo', 'Decompõe a duração total dos projetos para mostrar quanto tempo, em média, é gasto em cada fase principal (Desenvolvimento, Teste, Operações).', filename)

# --- 3. FINALIZAÇÃO E EXPORTAÇÃO ---
with open(report_path_analysis, 'a', encoding='utf-8') as f: f.write("</div></body></html>")
zip_filename_eda = 'Definitive_Analysis_Dashboard_FINAL.zip'
shutil.make_archive(zip_filename_eda.replace('.zip', ''), 'zip', report_dir_analysis)
print(f"\n--- Painel de Análise compactado em '{zip_filename_eda}' ---")

# --- CÓDIGO REATIVADO PARA GOOGLE COLAB ---
from google.colab import files
files.download(zip_filename_eda)
print(f"O download de '{zip_filename_eda}' deve começar em breve.")

print("\n✅ CÉLULA 2 CONCLUÍDA: Painel de Análise Definitivo gerado e exportado.")

--- Executando Célula 2: Geração do Painel de Análise Completo ---
Plot 'plot_01_performance_matrix.png' salvo.
Plot 'plot_02_case_durations_boxplot.png' salvo.


/tmp/ipython-input-3146869732.py:109: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  throughput_per_case = log_df_final.groupby("case:concept:name").apply(compute_avg_throughput).reset_index(name="avg_throughput_seconds")


Plot 'plot_03_lead_time_hist.png' salvo.
Plot 'plot_04_throughput_hist.png' salvo.
Plot 'plot_05_throughput_boxplot.png' salvo.
Plot 'plot_06_lead_time_vs_throughput.png' salvo.
Plot 'plot_07_activity_service_times.png' salvo.
Plot 'plot_08_top_handoffs.png' salvo.
Plot 'plot_09_top_handoffs_cost.png' salvo.
Plot 'plot_10_top_activities_plot.png' salvo.
Plot 'plot_11_resource_workload.png' salvo.
Plot 'plot_12_resource_avg_events.png' salvo.
Plot 'plot_13_resource_activity_matrix.png' salvo.
Plot 'plot_14_resource_handoffs.png' salvo.
Plot 'plot_15_cost_by_resource_type.png' salvo.
Plot 'plot_16_variants_frequency.png' salvo.
Plot 'plot_17_delay_by_teamsize.png' salvo.
Plot 'plot_18_median_duration_by_teamsize.png' salvo.
Plot 'plot_19_weekly_efficiency.png' salvo.
Plot 'plot_20_bottleneck_by_resource.png' salvo.
Plot 'plot_21_service_vs_wait_stacked.png' salvo.
Plot 'plot_22_wait_vs_service_scatter.png' salvo.
Plot 'plot_23_wait_time_evolution.png' salvo.
Plot 'plot_24_handoff_matrix_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

O download de 'Definitive_Analysis_Dashboard_FINAL.zip' deve começar em breve.

✅ CÉLULA 2 CONCLUÍDA: Painel de Análise Definitivo gerado e exportado.


In [30]:
# --- INÍCIO DO SCRIPT UNIFICADO: Dashboard Completo de Análise de Processos e Process Mining (v16 - CORRIGIDO) ---
print("--- Executando o Script Unificado de Análise Completa (v16 - CORRIGIDO) ---")

# --- 1. SETUP GERAL DE IMPORTS E DIRETÓRIOS ---
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os
import shutil
import networkx as nx
from collections import Counter
from IPython.display import display
from zipfile import ZipFile

# Imports de PM4PY
import pm4py
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualizer
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_miner

# --- Setup de Diretórios ---
report_dir = 'Relatorio_Unificado_Analise_Processos'
plots_dir = os.path.join(report_dir, 'plots')
models_dir = os.path.join(report_dir, 'models')
data_dir = os.path.join(report_dir, 'data')
if os.path.exists(report_dir): shutil.rmtree(report_dir)
os.makedirs(plots_dir, exist_ok=True); os.makedirs(models_dir, exist_ok=True); os.makedirs(data_dir, exist_ok=True)

# --- Setup do Relatório HTML ---
report_path = os.path.join(report_dir, 'Relatorio_Unificado.html')
html_header = """
<!DOCTYPE html><html lang="pt"><head><meta charset="UTF-8"><title>Relatório Unificado de Análise de Processos</title><style>
body {{ font-family: sans-serif; margin: 0; background-color: #f4f4f9; color: #333; }}
.container {{ max-width: 1300px; margin: auto; padding: 20px; background-color: #fff; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
h1, h2, h3 {{ color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px; }}
h1 {{ font-size: 2.5em; text-align: center; }} h2 {{ font-size: 2em; margin-top: 40px; }} h3 {{ font-size: 1.5em; border-bottom: 1px solid #ccc; }}
img {{ max-width: 90%; height: auto; display: block; margin: 20px auto; border: 1px solid #ddd; border-radius: 5px; }}
p, li {{ line-height: 1.6; }} table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
th, td {{ padding: 12px; border: 1px solid #ddd; text-align: left; }} th {{ background-color: #3498db; color: white; }}
tr:nth-child(even) {{ background-color: #f2f2f2; }}
</style></head><body><div class="container">
<h1>Relatório Unificado de Análise de Processos</h1><p>Relatório gerado em: {date}</p>
""".format(date=pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'))
with open(report_path, 'w', encoding='utf-8') as f: f.write(html_header)
plot_counter = 0

# --- 2. FUNÇÕES DE APOIO GLOBAIS ---
def save_artefact_and_add_to_html_report(artefact, title, filename_base, artefact_type='plot', content_text=''):
    global plot_counter
    plot_counter += 1
    filename_base_counted = f"{plot_counter:02d}_{filename_base}"
    subfolder = 'models' if artefact_type == 'model' else 'plots'
    img_path_in_html = f"{subfolder}/{filename_base_counted}.png"
    directory = models_dir if artefact_type == 'model' else plots_dir
    full_save_path_png = os.path.join(directory, f"{filename_base_counted}.png")
    if 'graphviz' in str(type(artefact)):
        try:
            # Para objetos graphviz, usamos o render para salvar.
            artefact.render(full_save_path_png.replace('.png', ''), cleanup=True, format='png')
        except Exception:
             # Fallback para visualizadores pm4py que retornam objetos graphviz
            if artefact_type == 'model':
                pn_visualizer.save(artefact, full_save_path_png)
            else:
                dfg_visualizer.save(artefact, full_save_path_png)
    elif 'matplotlib' in str(type(artefact)):
        artefact.savefig(full_save_path_png, bbox_inches='tight')
    else:
        print(f"Não foi possível salvar o artefato do tipo: {type(artefact)}")

    plt.close('all')
    with open(report_path, 'a', encoding='utf-8') as f:
        f.write(f"<h3>{title}</h3>\n{content_text}\n<img src='{img_path_in_html}' alt='{title}'>\n")
    print(f"Artefacto '{filename_base_counted}.png' salvo.")

def calculate_model_metrics(log, petri_net, initial_marking, final_marking):
    print("A calcular métricas para o modelo...")
    fitness = replay_fitness_evaluator.apply(log, petri_net, initial_marking, final_marking, variant=replay_fitness_evaluator.Variants.TOKEN_BASED)
    precision = precision_evaluator.apply(log, petri_net, initial_marking, final_marking, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
    generalization = generalization_evaluator.apply(log, petri_net, initial_marking, final_marking)
    simplicity = simplicity_evaluator.apply(petri_net)
    metrics = {"Fitness": fitness.get('average_trace_fitness', 0), "Precisão": precision, "Generalização": generalization, "Simplicidade": simplicity}
    print(f"Métricas calculadas: {metrics}")
    return metrics

def plot_metrics_chart(metrics_dict, title):
    df_metrics = pd.DataFrame(list(metrics_dict.items()), columns=['Métrica', 'Valor'])
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.set_theme(style="whitegrid")
    barplot = sns.barplot(data=df_metrics, x='Métrica', y='Valor', palette='viridis', ax=ax, hue='Métrica', legend=False)
    for p in barplot.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.set_ylim(0, 1.05); ax.set_ylabel(''); ax.set_xlabel('')
    return fig

metrics_explanation_html = """
<p style='font-size:0.9em; color:#555;'><i><b>Legenda das Métricas:</b>
<ul>
    <li><b>Fitness:</b> Mede até que ponto o comportamento registado no log pode ser reproduzido pelo modelo.</li>
    <li><b>Precisão:</b> Mede se o modelo permite comportamentos que não são vistos no log.</li>
    <li><b>Generalização:</b> Mede se o modelo consegue generalizar o comportamento do log.</li>
    <li><b>Simplicidade:</b> Mede a complexidade do modelo. Modelos mais simples (valor mais alto) são mais fáceis de entender.</li>
</ul></i></p>
"""

# --- 3. CARREGAMENTO E PREPARAÇÃO CENTRALIZADA DOS DADOS ---
print("\n--- A carregar e preparar os dados base uma única vez ---")
try:
    df_projects_raw = pd.read_csv("projects.csv", parse_dates=['start_date', 'end_date', 'planned_end_date'])
    df_tasks_raw = pd.read_csv("tasks.csv", parse_dates=['start_date', 'end_date'])
    df_resources_raw = pd.read_csv("resources.csv")
    df_allocations_raw = pd.read_csv("resource_allocations.csv", parse_dates=['allocation_date'])
    df_dependencies_raw = pd.read_csv("dependencies.csv")
    df_projects, df_tasks, df_resources, df_allocations, df_dependencies = df_projects_raw.copy(), df_tasks_raw.copy(), df_resources_raw.copy(), df_allocations_raw.copy(), df_dependencies_raw.copy()

    for col in ['project_id', 'task_id', 'resource_id', 'allocation_id']:
        for df in [df_projects, df_tasks, df_resources, df_allocations, df_dependencies]:
             if col in df.columns: df[col] = df[col].astype(str)

    log_df_final = df_tasks.merge(df_allocations, on='task_id').merge(df_resources, on='resource_id')
    log_df_final.rename(columns={'project_id_x': 'case:concept:name', 'task_name': 'concept:name', 'allocation_date': 'time:timestamp', 'resource_name': 'org:resource'}, inplace=True)
    log_df_final['case:concept:name'] = log_df_final['case:concept:name'].astype(str)
    log_df_final['lifecycle:transition'] = 'complete'
    log_df_final = log_df_final.sort_values('time:timestamp')
    event_log_pm4py = pm4py.convert_to_event_log(log_df_final)

    df_projects['days_diff'] = (df_projects['end_date'] - df_projects['planned_end_date']).dt.days
    df_projects['actual_duration_days'] = (df_projects['end_date'] - df_projects['start_date']).dt.days
    df_allocations_costs = df_allocations.merge(df_resources[['resource_id', 'cost_per_hour']], on='resource_id')
    df_allocations_costs['cost_of_work'] = df_allocations_costs['hours_worked'] * df_allocations_costs['cost_per_hour']
    total_cost_per_project = df_allocations_costs.groupby('project_id')['cost_of_work'].sum().reset_index()
    df_projects = df_projects.merge(total_cost_per_project.rename(columns={'cost_of_work': 'total_actual_cost'}), on='project_id', how='left')
    df_projects['cost_diff'] = df_projects['total_actual_cost'] - df_projects['budget_impact']
    df_projects['cost_per_day'] = df_projects['total_actual_cost'] / df_projects['actual_duration_days'].replace(0, 1)
    df_projects['completion_month'] = df_projects['end_date'].dt.to_period('M').astype(str)
    if 'path_name' not in df_projects.columns: df_projects['path_name'] = 'Default'

    print("Dados carregados e log de eventos criado com sucesso.")
except FileNotFoundError as e:
    print(f"ERRO: Ficheiro não encontrado - {e}. Cancele a execução e verifique se os CSVs estão na pasta correta.")
    exit()

# --- SECÇÃO 1: PAINEL DE KPIS E ANÁLISE DE ALTO NÍVEL ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 1: Painel de KPIs e Análise de Alto Nível</h2>")
kpi_data = {'Métrica': ['Total de Projetos', 'Total de Tarefas', 'Total de Recursos', 'Duração Média (dias)', 'Custo Médio'],
             'Valor': [df_projects['project_id'].nunique(), df_tasks['task_id'].nunique(), df_resources['resource_id'].nunique(), df_projects['actual_duration_days'].mean(), df_projects['total_actual_cost'].mean()]}
kpi_df = pd.DataFrame(kpi_data)
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h3>Painel de KPIs de Alto Nível</h3>" + kpi_df.style.format({'Valor': '{:,.2f}'}).to_html(index=False))
fig_perf, ax_perf = plt.subplots(figsize=(12, 8))
sns.scatterplot(data=df_projects, x='days_diff', y='cost_diff', hue='path_name', s=100, alpha=0.7, ax=ax_perf)
ax_perf.axhline(0, color='black', linestyle='--'); ax_perf.axvline(0, color='black', linestyle='--')
save_artefact_and_add_to_html_report(fig_perf, "Matriz de Performance: Prazo vs. Orçamento", "performance_matrix")

# --- SECÇÃO 2: DESCOBERTA E AVALIAÇÃO DE MODELOS DE PROCESSO ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 2: Descoberta e Avaliação de Modelos de Processo</h2>")

variants_dict = variants_filter.get_variants(event_log_pm4py)
top_variants_list = sorted(variants_dict.items(), key=lambda x: len(x[1]), reverse=True)[:3]
top_variant_names = [v[0] for v in top_variants_list]
log_top_3_variants = variants_filter.apply(event_log_pm4py, top_variant_names)
pt_inductive = inductive_miner.apply(log_top_3_variants)
net_im, im_im, fm_im = pt_converter.apply(pt_inductive)
gviz_im = pn_visualizer.apply(net_im, im_im, fm_im)
save_artefact_and_add_to_html_report(gviz_im, 'Modelo de Processo (Inductive Miner)', 'model_inductive_petrinet', artefact_type='model', content_text='<p><b>Parâmetros:</b> Log filtrado para as Top 3 variantes mais comuns.</p>')
metrics_im = calculate_model_metrics(log_top_3_variants, net_im, im_im, fm_im)
fig_metrics_im = plot_metrics_chart(metrics_im, 'Métricas de Qualidade (Inductive Miner)')
save_artefact_and_add_to_html_report(fig_metrics_im, 'Gráfico de Métricas (Inductive Miner)', 'metrics_inductive', content_text=metrics_explanation_html)

net_hm, im_hm, fm_hm = heuristics_miner.apply(log_top_3_variants, parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.5})
gviz_hm = pn_visualizer.apply(net_hm, im_hm, fm_hm)
save_artefact_and_add_to_html_report(gviz_hm, 'Modelo de Processo (Heuristics Miner)', 'model_heuristic_petrinet', artefact_type='model', content_text=f'<p><b>Parâmetros:</b> Dependency Threshold = 0.5.</p>')
metrics_hm = calculate_model_metrics(log_top_3_variants, net_hm, im_hm, fm_hm)
fig_metrics_hm = plot_metrics_chart(metrics_hm, 'Métricas de Qualidade (Heuristics Miner)')
save_artefact_and_add_to_html_report(fig_metrics_hm, 'Gráfico de Métricas (Heuristics Miner)', 'metrics_heuristic', content_text=metrics_explanation_html)

# --- SECÇÃO 3: ANÁLISE DE PERFORMANCE E TEMPO DE CICLO (AVANÇADA) ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 3: Análise de Performance e Tempo de Ciclo (Avançada)</h2>")
kpi_temporal = df_projects.groupby('completion_month').agg(avg_lead_time=('actual_duration_days', 'mean'), throughput=('project_id', 'count')).reset_index()
fig_kpi, ax1 = plt.subplots(figsize=(15, 7))
ax1.plot(kpi_temporal['completion_month'], kpi_temporal['avg_lead_time'], marker='o', color='b', label='Lead Time Médio (dias)')
ax1.set_ylabel('Dias', color='b'); ax1.tick_params(axis='y', labelcolor='b'); ax1.tick_params(axis='x', rotation=45)
ax2 = ax1.twinx()
ax2.bar(kpi_temporal['completion_month'], kpi_temporal['throughput'], color='g', alpha=0.6, label='Throughput (Projetos Concluídos)')
ax2.set_ylabel('Nº de Projetos', color='g'); ax2.tick_params(axis='y', labelcolor='g')
fig_kpi.suptitle('Séries Temporais de KPIs de Performance')
save_artefact_and_add_to_html_report(fig_kpi, "Séries Temporais de KPI", "kpi_time_series")

# Gráfico de Gantt - CORRIGIDO para incluir todos os projetos
def generate_custom_gantt_chart_all(df_tasks, df_projects):
    fig_gantt, ax_gantt = plt.subplots(figsize=(20, len(df_projects) * 0.5)) # Ajusta o tamanho dinamicamente
    all_projects = df_projects.sort_values('start_date')['project_id'].tolist()
    gantt_data = df_tasks[df_tasks['project_id'].isin(all_projects)].sort_values(['project_id', 'start_date'])

    project_y_map = {proj_id: i for i, proj_id in enumerate(all_projects)}

    task_colors = plt.get_cmap('viridis', gantt_data['task_name'].nunique())
    color_map = {task_name: task_colors(i) for i, task_name in enumerate(gantt_data['task_name'].unique())}

    for _, task in gantt_data.iterrows():
        y_pos = project_y_map[task['project_id']]
        ax_gantt.barh(y_pos, (task['end_date'] - task['start_date']).days + 1, left=task['start_date'], height=0.6, color=color_map[task['task_name']], edgecolor='black')

    ax_gantt.set_yticks(list(project_y_map.values()))
    ax_gantt.set_yticklabels([f"Projeto {pid}" for pid in project_y_map.keys()])
    ax_gantt.invert_yaxis()

    ax_gantt.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)

    handles = [plt.Rectangle((0,0),1,1, color=color_map[label]) for label in color_map]
    ax_gantt.legend(handles, color_map.keys(), title='Tipo de Tarefa', bbox_to_anchor=(1.05, 1), loc='upper left')

    ax_gantt.set_title('Linha do Tempo de Todos os Projetos (Gantt Chart)')
    ax_gantt.set_xlabel("Linha do Tempo")
    ax_gantt.set_ylabel("Projeto")

    fig_gantt.tight_layout()
    return fig_gantt

fig_gantt = generate_custom_gantt_chart_all(df_tasks, df_projects)
save_artefact_and_add_to_html_report(fig_gantt, "Linha do Tempo de Todos os Projetos (Gantt Chart)", "gantt_chart_all_projects")


# --- SECÇÃO 4: ANÁLISE DE GARGALOS ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 4: Análise de Gargalos</h2>")
df_tasks_sorted = df_tasks.sort_values(['project_id', 'start_date'])
df_tasks_sorted['previous_end_date'] = df_tasks_sorted.groupby('project_id')['end_date'].shift(1)
df_tasks_sorted['waiting_time_days'] = (df_tasks_sorted['start_date'] - df_tasks_sorted['previous_end_date']).dt.total_seconds() / (24 * 3600)
df_tasks_sorted.loc[df_tasks_sorted['waiting_time_days'] < 0, 'waiting_time_days'] = 0
df_tasks_sorted['previous_task_name'] = df_tasks_sorted.groupby('project_id')['task_name'].shift(1)
df_tasks_sorted.dropna(subset=['previous_task_name'], inplace=True)
df_tasks_sorted['transition'] = df_tasks_sorted['previous_task_name'] + ' -> ' + df_tasks_sorted['task_name']
gargalos = df_tasks_sorted.groupby('transition')['waiting_time_days'].mean().sort_values(ascending=False).head(15)
fig_garg, ax_garg = plt.subplots(figsize=(12, 8))
gargalos.sort_values().plot(kind='barh', ax=ax_garg, color='tomato')
save_artefact_and_add_to_html_report(fig_garg, "Ranking de Gargalos (Top 15 Transições com Maior Tempo de Espera)", "bottleneck_ranking_adv")

dfg_perf, start_activities, end_activities = pm4py.discover_performance_dfg(event_log_pm4py)
gviz_dfg = dfg_visualizer.apply(dfg_perf, log=event_log_pm4py, variant=dfg_visualizer.Variants.PERFORMANCE)
save_artefact_and_add_to_html_report(gviz_dfg, "Heatmap de Performance no Processo", "performance_heatmap", artefact_type='model', content_text="Diagrama de processo onde a cor das arestas representa o tempo médio de transição.")

# Heatmap Temporal de Ocorrências de Atividades - CORRIGIDO
fig_hmt, ax_hmt = plt.subplots(figsize=(10, 6))
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
log_df_final['weekday'] = log_df_final['time:timestamp'].dt.day_name()
heatmap_data = log_df_final.groupby('weekday')['case:concept:name'].count().reindex(weekday_order).fillna(0)
sns.barplot(x=heatmap_data.index, y=heatmap_data.values, palette='viridis', ax=ax_hmt, hue=heatmap_data.index, legend=False)
ax_hmt.set_title('Ocorrências de Atividades por Dia da Semana')
ax_hmt.set_xlabel('Dia da Semana')
ax_hmt.set_ylabel('Número de Ocorrências')
plt.xticks(rotation=45, ha='right')
fig_hmt.tight_layout()
save_artefact_and_add_to_html_report(fig_hmt, "Ocorrências de Atividades por Dia da Semana", "temporal_heatmap_fixed", content_text="Este gráfico mostra a distribuição do volume de trabalho ao longo da semana, identificando dias de maior atividade.")


# --- SECÇÃO 5: ANÁLISE DE RECURSOS ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 5: Análise de Recursos</h2>")
handovers = {}
for _, group in log_df_final.groupby('case:concept:name'):
    resources = group.sort_values('time:timestamp')['org:resource'].tolist()
    for i in range(len(resources) - 1):
        if resources[i] != resources[i+1]:
            pair = (resources[i], resources[i+1])
            handovers[pair] = handovers.get(pair, 0) + 1
fig_net, ax_net = plt.subplots(figsize=(14, 14))
G = nx.DiGraph()
for (source, target), weight in handovers.items(): G.add_edge(source, target, weight=weight)
pos = nx.spring_layout(G, k=0.9, iterations=50, seed=42)
weights = [G[u][v]['weight'] for u,v in G.edges()]
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=3000, edge_color='gray', width=[w*0.5 for w in weights], ax=ax_net, font_size=10, connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'weight'), ax=ax_net)
save_artefact_and_add_to_html_report(fig_net, "Rede Social de Recursos (Handover Network)", "resource_network_adv")

perf_recursos = df_allocations.groupby('resource_id').agg(total_hours=('hours_worked', 'sum'), total_tasks=('task_id', 'nunique')).reset_index()
perf_recursos['avg_hours_per_task'] = perf_recursos['total_hours'] / perf_recursos['total_tasks']
perf_recursos = perf_recursos.merge(df_resources[['resource_id', 'skill_level', 'resource_name']], on='resource_id')
fig_skill, ax_skill = plt.subplots(figsize=(10, 7))
sns.regplot(data=perf_recursos, x='skill_level', y='avg_hours_per_task', ax=ax_skill)
save_artefact_and_add_to_html_report(fig_skill, "Gráfico de Skills vs. Performance", "skill_vs_performance_adv", content_text="Relação entre o nível de skill e a performance (média de horas por tarefa).")

# --- SECÇÃO 6: NOVOS GRÁFICOS E ANÁLISES ADICIONAIS ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("<h2>Secção 6: Novas Análises e Visualizações</h2>")
print("\n--- A gerar os 12 gráficos adicionais solicitados ---")

# 1. Gráfico de Variantes de Processo com Duração Média
variants_df = log_df_final.groupby('case:concept:name').agg(
    variant=('concept:name', lambda x: tuple(x)),
    start_timestamp=('time:timestamp', 'min'),
    end_timestamp=('time:timestamp', 'max')
).reset_index()
variants_df['duration_hours'] = (variants_df['end_timestamp'] - variants_df['start_timestamp']).dt.total_seconds() / 3600
variant_durations = variants_df.groupby('variant').agg(
    count=('case:concept:name', 'count'),
    avg_duration_hours=('duration_hours', 'mean')
).reset_index().sort_values(by='count', ascending=False).head(10)
variant_durations['variant_str'] = variant_durations['variant'].apply(lambda x: ' -> '.join([str(i) for i in x])).astype(str)
for col in variant_durations.columns:
    variant_durations[col] = variant_durations[col].apply(lambda x: str(x))
fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='avg_duration_hours', y='variant_str', data=variant_durations, palette='plasma', ax=ax, hue='variant_str', legend=False)
ax.set_title('Duração Média das 10 Variantes de Processo Mais Comuns')
ax.set_xlabel('Duração Média (horas)')
ax.set_ylabel('Variante do Processo')
fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Gráfico de Variantes de Processo com Duração Média", "variant_duration_plot")

# 3. Diagrama de Dispersão de Desvios
aligned_traces = alignments_miner.apply(event_log_pm4py, net_im, im_im, fm_im)
deviations_list = [{'fitness': trace['fitness'], 'deviations': sum(1 for move in trace['alignment'] if '>>' in move[0] or '>>' in move[1])} for trace in aligned_traces]
deviations_df = pd.DataFrame(deviations_list)
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x='fitness', y='deviations', data=deviations_df, alpha=0.6, ax=ax)
ax.set_title('Diagrama de Dispersão de Desvios (Fitness vs. Desvios)')
ax.set_xlabel('Fitness do Caso'); ax.set_ylabel('Número de Desvios')
fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Diagrama de Dispersão de Desvios", "deviation_scatter_plot")

# 4. Score de Conformidade ao Longo do Tempo
case_fitness_data = [{'project_id': str(trace.attributes['concept:name']), 'fitness': alignment['fitness']} for trace, alignment in zip(event_log_pm4py, aligned_traces)]
case_fitness_df = pd.DataFrame(case_fitness_data)
case_fitness_df = case_fitness_df.merge(df_projects[['project_id', 'end_date']], on='project_id')
case_fitness_df['end_month'] = case_fitness_df['end_date'].dt.to_period('M').astype(str)
monthly_fitness = case_fitness_df.groupby('end_month')['fitness'].mean().reset_index()
monthly_fitness['end_month'] = monthly_fitness['end_month'].astype(str)
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=monthly_fitness, x='end_month', y='fitness', marker='o', ax=ax)
ax.set_title('Score de Conformidade ao Longo do Tempo')
ax.set_xlabel('Mês de Conclusão'); ax.set_ylabel('Fitness Médio'); ax.set_ylim(0, 1.05)
ax.tick_params(axis='x', rotation=45); fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Score de Conformidade ao Longo do Tempo", "conformance_over_time_plot")

# 5. Séries Temporais de KPI (Custo por Dia)
fig, ax = plt.subplots(figsize=(12, 6))
df_projects_kpi = df_projects.copy()
df_projects_kpi['completion_date'] = pd.to_datetime(df_projects_kpi['end_date'].dt.date)
kpi_daily = df_projects_kpi.groupby('completion_date').agg(avg_cost_per_day=('cost_per_day', 'mean')).reset_index()
kpi_daily['completion_date'] = pd.to_datetime(kpi_daily['completion_date'])
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=kpi_daily, x='completion_date', y='avg_cost_per_day', ax=ax)
ax.set_title('Custo Médio por Dia ao Longo do Tempo')
ax.set_xlabel('Data de Conclusão'); ax.set_ylabel('Custo Médio por Dia'); fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Séries Temporais de KPI (Custo por Dia)", "cost_per_day_time_series")

# 6. Gráfico Acumulado de Throughput
df_projects_sorted = df_projects.sort_values(by='end_date')
df_projects_sorted['cumulative_throughput'] = range(1, len(df_projects_sorted) + 1)
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(x='end_date', y='cumulative_throughput', data=df_projects_sorted, ax=ax)
ax.set_title('Gráfico Acumulado de Throughput')
ax.set_xlabel('Data de Conclusão'); ax.set_ylabel('Número Acumulado de Projetos'); fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Gráfico Acumulado de Throughput", "cumulative_throughput_plot")

# 7. Visualização de Sequência de Variantes
def generate_custom_variants_plot(event_log):
    variants = variants_filter.get_variants(event_log)
    top_variants = sorted(variants.items(), key=lambda item: len(item[1]), reverse=True)[:10]
    variant_sequences = {str(k): [str(a) for a in k] for k, v in top_variants}
    fig, ax = plt.subplots(figsize=(15, 10))
    colors = plt.get_cmap('tab10', len(variant_sequences))
    all_activities = sorted(list(set([act for seq in variant_sequences.values() for act in seq])))
    activity_to_y = {activity: i for i, activity in enumerate(all_activities)}
    for i, (variant_name, sequence) in enumerate(variant_sequences.items()):
        y_values = [activity_to_y[activity] for activity in sequence]
        x_values = range(len(sequence))
        ax.plot(x_values, y_values, marker='o', linestyle='-', label=f"Variante {i+1} ({len(top_variants[i][1])} casos)", color=colors(i))
    max_len = max(len(s) for s in variant_sequences.values()) if variant_sequences else 0
    ax.set_xticks(range(max_len))
    ax.set_xticklabels([f'Etapa {i+1}' for i in range(max_len)], rotation=45, ha='right')
    ax.set_yticks(list(activity_to_y.values()))
    ax.set_yticklabels(list(activity_to_y.keys()))
    ax.set_title('Sequência de Atividades das 10 Variantes Mais Comuns')
    ax.set_xlabel('Etapas do Processo'); ax.set_ylabel('Atividade')
    ax.legend(title='Variantes', loc='upper left', bbox_to_anchor=(1, 1))
    fig.tight_layout()
    return fig

new_fig_pc = generate_custom_variants_plot(event_log_pm4py)
save_artefact_and_add_to_html_report(new_fig_pc, "Sequência de Atividades das Variantes (Visualização Personalizada)", "custom_variants_sequence_plot", content_text="Este gráfico mostra a sequência de atividades das 10 variantes de processo mais comuns.")

# 8. Análise de Tempo entre Marcos
milestones = ['Analise e Design', 'Implementacao da Funcionalidade', 'Execucao de Testes', 'Deploy da Aplicacao']
df_milestones = df_tasks_raw[df_tasks_raw['task_name'].isin(milestones)].copy()
milestone_pairs = []
for project_id, group in df_milestones.groupby('project_id'):
    sorted_tasks = group.sort_values('start_date')
    for i in range(len(sorted_tasks) - 1):
        start_task, end_task = sorted_tasks.iloc[i], sorted_tasks.iloc[i+1]
        duration = (end_task['start_date'] - start_task['end_date']).total_seconds() / 3600
        if duration >= 0: milestone_pairs.append({'transition': str(start_task['task_name']) + ' -> ' + str(end_task['task_name']), 'duration_hours': duration})
df_milestone_pairs = pd.DataFrame(milestone_pairs)

if not df_milestone_pairs.empty:
    df_milestone_pairs['transition'] = df_milestone_pairs['transition'].astype(str)
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.boxplot(data=df_milestone_pairs, x='duration_hours', y='transition', ax=ax, hue='transition', legend=False, orient='h')
    ax.set_title('Análise de Tempo entre Marcos do Processo'); ax.set_ylabel('Transição de Marcos'); ax.set_xlabel('Tempo de Espera (horas)'); fig.tight_layout()
    save_artefact_and_add_to_html_report(fig, "Frame Time Analysis (Análise de Tempo entre Marcos)", "milestone_time_analysis_plot")

# 9. Matriz de Tempo de Espera
df_tasks_sorted['previous_task_name'] = df_tasks_sorted['previous_task_name'].astype(str)
df_tasks_sorted['task_name'] = df_tasks_sorted['task_name'].astype(str)
waiting_times_matrix = df_tasks_sorted.pivot_table(index='previous_task_name', columns='task_name', values='waiting_time_days', aggfunc='mean').fillna(0)
waiting_times_matrix.index = waiting_times_matrix.index.map(str)
waiting_times_matrix.columns = waiting_times_matrix.columns.map(str)
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(waiting_times_matrix * 24, cmap='YlGnBu', annot=True, fmt='.2f', linewidths=.5, ax=ax)
ax.set_title('Matriz de Tempo de Espera entre Atividades (horas)'); ax.set_xlabel('Próxima Atividade'); ax.set_ylabel('Atividade Anterior'); fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Matriz de Tempo de Espera entre Atividades", "waiting_time_matrix_plot")

# 10. Métricas de Eficiência Individual por Recurso
resource_efficiency = log_df_final.groupby('org:resource').agg(total_hours_worked=('hours_worked', 'sum'), total_tasks_completed=('concept:name', 'count')).reset_index()
resource_efficiency['avg_hours_per_task'] = resource_efficiency['total_hours_worked'] / resource_efficiency['total_tasks_completed']
resource_efficiency['org:resource'] = resource_efficiency['org:resource'].astype(str)
fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(data=resource_efficiency.sort_values(by='avg_hours_per_task'), x='avg_hours_per_task', y='org:resource', palette='magma', ax=ax, hue='org:resource', legend=False, orient='h')
ax.set_title('Métricas de Eficiência Individual por Recurso'); ax.set_ylabel('Recurso'); ax.set_xlabel('Média de Horas por Tarefa'); fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Métricas de Eficiência Individual por Recurso", "resource_efficiency_plot")

# 11. Histograma de Tempo de Espera por Atividade - ALTERADO PARA GRÁFICO DE BARRAS
df_tasks_sorted['sojourn_time_hours'] = (df_tasks_sorted['start_date'] - df_tasks_sorted['previous_end_date']).dt.total_seconds() / 3600
df_tasks_sorted.loc[df_tasks_sorted['sojourn_time_hours'] < 0, 'sojourn_time_hours'] = 0
waiting_time_by_task = df_tasks_sorted.groupby('task_name')['sojourn_time_hours'].mean().reset_index()
fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(data=waiting_time_by_task.sort_values(by='sojourn_time_hours', ascending=False), x='sojourn_time_hours', y='task_name', ax=ax, palette='viridis', hue='task_name', legend=False)
ax.set_title('Tempo Médio de Espera por Atividade')
ax.set_xlabel('Tempo de Espera Médio (horas)')
ax.set_ylabel('Atividade')
fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Tempo Médio de Espera por Atividade", "avg_waiting_time_by_activity_plot", content_text="Este gráfico mostra o tempo médio de espera entre a conclusão da tarefa anterior e o início da tarefa atual, para cada tipo de atividade.")


# --- 12. Rede de Recursos (Resource Network) - Corrigido para Grafo Bipartido ---
print("Gerando a Rede de Recursos por Função (Grafo Bipartido)...")
if 'org:role' not in log_df_final.columns:
    # Simula a coluna de funções se ela não existir
    log_df_final['org:role'] = df_resources.set_index('resource_id').loc[log_df_final['resource_id'], 'skill_level'].values
    log_df_final['org:role'] = log_df_final['org:role'].fillna('N/A')

resource_role_counts = log_df_final.groupby(['org:resource', 'org:role']).size().reset_index(name='count')
G_bipartite = nx.Graph()
resources = resource_role_counts['org:resource'].unique()
roles = resource_role_counts['org:role'].unique()
G_bipartite.add_nodes_from(resources, bipartite=0)
G_bipartite.add_nodes_from(roles, bipartite=1)

for _, row in resource_role_counts.iterrows():
    G_bipartite.add_edge(row['org:resource'], row['org:role'], weight=row['count'])

pos = nx.bipartite_layout(G_bipartite, resources, align='vertical') # Alinhamento para melhor visualização
fig, ax = plt.subplots(figsize=(15, 12))

nx.draw_networkx_nodes(G_bipartite, pos, nodelist=resources, node_color='skyblue', node_size=3000, ax=ax)
nx.draw_networkx_nodes(G_bipartite, pos, nodelist=roles, node_color='lightgreen', node_size=3000, ax=ax)

edges = G_bipartite.edges(data=True)
weights = [d['weight'] for u, v, d in edges]
nx.draw_networkx_edges(G_bipartite, pos, width=[w * 0.1 for w in weights], edge_color='gray', ax=ax)

# Corrigido: Mapeia os rótulos de forma correta
node_labels = {node: node for node in G_bipartite.nodes()}
nx.draw_networkx_labels(G_bipartite, pos, labels=node_labels, font_size=10)

edge_labels = {(u, v): d['weight'] for u, v, d in G_bipartite.edges(data=True)}
nx.draw_networkx_edge_labels(G_bipartite, pos, edge_labels=edge_labels, font_size=9)

ax.set_title('Rede de Recursos por Função (Grafo Bipartido)')
ax.text(0.5, 1.05, 'Azul: Recursos | Verde: Funções/Skills', transform=ax.transAxes, ha='center', fontsize=12, color='gray') # Adiciona legenda de cores
ax.axis('off')
fig.tight_layout()
save_artefact_and_add_to_html_report(fig, "Rede de Recursos por Função", "resource_network_bipartite", content_text="<p>Este gráfico mostra a relação entre os recursos (nós azuis) e as suas funções/níveis de skill (nós verdes). A espessura e o número sobre as linhas indicam o volume de trabalho de um recurso para uma função específica.</p>")

# --- 7. FINALIZAÇÃO E EXPORTAÇÃO ---
with open(report_path, 'a', encoding='utf-8') as f: f.write("</div></body></html>")
zip_filename = 'Relatorio_Unificado_Analise_Processos.zip'
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', report_dir)
from google.colab import files
zip_filename = 'Relatorio_Unificado_Analise_Processos.zip'
files.download(zip_filename)
print(f"\n--- Relatório Unificado compactado em '{zip_filename}' ---")
print(f"O ficheiro '{zip_filename}' foi gerado com sucesso no diretório.")
print("\n✅ SCRIPT UNIFICADO CONCLUÍDO: Relatório completo gerado e exportado.")

--- Executando o Script Unificado de Análise Completa (v16 - CORRIGIDO) ---

--- A carregar e preparar os dados base uma única vez ---
Dados carregados e log de eventos criado com sucesso.
Artefacto '01_performance_matrix.png' salvo.
Artefacto '02_model_inductive_petrinet.png' salvo.
A calcular métricas para o modelo...


replaying log with TBR, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/23 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

Métricas calculadas: {'Fitness': 1.0, 'Precisão': 0.6285714285714286, 'Generalização': 0.5045961255296935, 'Simplicidade': 0.7464788732394366}
Artefacto '03_metrics_inductive.png' salvo.
Artefacto '04_model_heuristic_petrinet.png' salvo.
A calcular métricas para o modelo...


...

aligning log, completed variants ::   0%|          | 0/43 [00:00<?, ?it/s]

Artefacto '14_deviation_scatter_plot.png' salvo.
Artefacto '15_conformance_over_time_plot.png' salvo.
Artefacto '16_cost_per_day_time_series.png' salvo.
Artefacto '17_cumulative_throughput_plot.png' salvo.
Artefacto '18_custom_variants_sequence_plot.png' salvo.
Artefacto '19_milestone_time_analysis_plot.png' salvo.
Artefacto '20_waiting_time_matrix_plot.png' salvo.
Artefacto '21_resource_efficiency_plot.png' salvo.
Artefacto '22_avg_waiting_time_by_activity_plot.png' salvo.
Gerando a Rede de Recursos por Função (Grafo Bipartido)...
Artefacto '23_resource_network_bipartite.png' salvo.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Relatório Unificado compactado em 'Relatorio_Unificado_Analise_Processos.zip' ---
O ficheiro 'Relatorio_Unificado_Analise_Processos.zip' foi gerado com sucesso no diretório.

✅ SCRIPT UNIFICADO CONCLUÍDO: Relatório completo gerado e exportado.
